In [10]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("../data/raw.csv")
df.head() 

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [9]:
df["readmit_30"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)
df["readmit_30"].value_counts()

readmit_30
0    90409
1    11357
Name: count, dtype: int64

In [11]:
df.isna().sum().sort_values(ascending=False)

max_glu_serum               96420
A1Cresult                   84748
encounter_id                    0
chlorpropamide                  0
acetohexamide                   0
glipizide                       0
glyburide                       0
tolbutamide                     0
pioglitazone                    0
rosiglitazone                   0
acarbose                        0
miglitol                        0
troglitazone                    0
tolazamide                      0
examide                         0
citoglipton                     0
insulin                         0
glyburide-metformin             0
glipizide-metformin             0
glimepiride-pioglitazone        0
metformin-rosiglitazone         0
metformin-pioglitazone          0
change                          0
diabetesMed                     0
readmitted                      0
glimepiride                     0
repaglinide                     0
nateglinide                     0
patient_nbr                     0
race          

In [ ]:
# Handle missing values and separete features
categorical = df.select_dtypes(include=["object"]).columns
numerical = df.select_dtypes(exclude=["object"]).columns

print("Categorical: ", categorical)
print("Numerical: ", numerical)

Categorical:  Index(['race', 'gender', 'age', 'weight', 'payer_code', 'medical_specialty',
       'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed', 'readmitted'],
      dtype='object')
Numerical:  Index(['encounter_id', 'patient_nbr', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient',
       'number_diagnoses', 'readmit_30'],
      dtype='object')

In [ ]:
# Handle missing values
df = df.drop(columns=["encounter_id", "patient_nbr"])

In [ ]:
# Replace "?" with NaN and forward fill
df.replace("?", pd.NA, inplace=True)
df.fillna(method="ffill", inplace=True)

/var/folders/2g/fnln3gmx15z1g9xv8r0swrhm0000gn/T/ipykernel_6634/3818069399.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", inplace=True)


In [16]:
# One-hot encode categorical features
df = pd.get_dummies(df, columns=categorical, drop_first=True)

In [ ]:
# Normalise numerical features
numerical = df.select_dtypes(exclude="object").columns

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])

In [19]:
# Split into train/test
from sklearn.model_selection import train_test_split

X = df.drop("readmit_30", axis=1)
y = df["readmit_30"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)